In [1]:
# general
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
import os

# custom local libs
from function import baseball_lambda

/Users/rosclay/Documents/VSCode/langgraph-multi-agent/venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `bind_tools` is in beta. It is actively being worked on, so the API may change.
  warn_beta(
Python REPL can execute arbitrary code. Use with caution.


In [2]:
# read local .env file
_ = load_dotenv(find_dotenv()) 

In [3]:
# set Langsmith project
today = datetime.now().strftime("%Y%m%d")
os.environ["LANGCHAIN_PROJECT"] = f"Baseball Curveballs - {today}"

In [4]:
session_id = '2110956'

In [5]:
task =  "Consider the first week of August 2020 - find the names of 3 pitchers who's curveballs were most similar to Max Scherzer's."
result = baseball_lambda.execute_workflow(task, session_id)
print(result)


*** Entered Orchestrate Node ***

*** Entered Retrieve Node ***

Distance to neareast plan: 0.572

*** Entered Initialize Node ***

Formulating a new plan to solve the task.

*** Entered Update Node ***

Updating plan based function documentation
Here is the updated plan to find the 3 pitchers with curveballs most similar to Max Scherzer's in the first week of August 2020:

1. Use the playerid_lookup function from the pybaseball library to get Max Scherzer's MLBAM ID by passing his last name "Scherzer" and first name "Max". 

2. Use the statcast_pitcher function from the pybaseball library to get Scherzer's statcast data from 2020-08-01 to 2020-08-07, filtering for only pitches where pitch_type = 'CU'. 

3. Calculate the average release_speed, release_spin_rate, pfx_x, and pfx_z for Scherzer's curveball from the data in step 2.

4. Use the statcast function from the pybaseball library to get all pitches thrown between 2020-08-01 and 2020-08-07 where pitch_type = 'CU'.

5. Group the da

In [6]:
task = """
make sure the plan follows this general flow:
1) create an average vector for each pitcher's curveball.
2) Train a knn model on this data
3) Use this model to find the 3 pitchers with the most similar curveball to Max Scherzer
"""
result = baseball_lambda.execute_workflow(task, session_id)
print(result)


*** Entered Orchestrate Node ***

*** Entered Revise Node ***

The updated plan looks good and covers the key steps to identify the 3 pitchers with the most similar curveballs to Max Scherzer's during the specified time period. The plan includes:

1. Retrieving Scherzer's MLBAM ID
2. Retrieving all curveballs thrown by pitchers between 2020-08-01 and 2020-08-07
3. Creating a dataframe with key curveball metrics for each pitcher
4. Scaling the data and calculating Scherzer's average curveball metrics
5. Training a KNeighborsClassifier model to find the 3 pitchers with the most similar curveball metrics to Scherzer's
6. Retrieving the names of those 3 pitchers

This plan should allow us to effectively identify the 3 pitchers with the most similar curveballs to Scherzer's during that time period. Let me know if you have any other feedback or if you would like me to modify the plan further.

Are you satisfied with this plan?


In [7]:
task = 'approved'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)


*** Entered Orchestrate Node ***
Unknown plan. Executing with Opus

*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
from pybaseball import playerid_lookup, statcast_pitcher
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Get Max Scherzer's MLBAM ID
scherzer_id = playerid_lookup('Scherzer', 'Max')['key_mlbam'][0]
print(f"Max Scherzer's MLBAM ID is: {scherzer_id}")

Result:
Gathering player lookup table. This may take a moment.
Max Scherzer's MLBAM ID is: 453286


*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
# Get all pitches thrown between 2020-08-01 and 2020-08-07
data = statcast('2020-08-01', '2020-08-07') 

# Filter to only curveballs
curveballs = data.loc[data['pitch_type'] == 'CU']
print(f"Retrieved {len(curveballs)} curveballs between 2020-08-01 and 2020-08-07")

Result:
NameError("name 'statcast' is not defined")

*** Entered Generate Node ***



  0%|          | 0/7 [00:00<?, ?it/s]


*** Entered Execute Node ***


Executing:
from pybaseball import playerid_lookup, statcast_pitcher, statcast
import pandas as pd
from sklearn.preprocessing import StandardScaler  
from sklearn.neighbors import NearestNeighbors

# Get Max Scherzer's MLBAM ID
scherzer_id = playerid_lookup('Scherzer', 'Max')['key_mlbam'][0]

# Get all pitches thrown between 2020-08-01 and 2020-08-07 
data = statcast('2020-08-01', '2020-08-07')

# Filter to only curveballs
curveballs = data.loc[data['pitch_type'] == 'CU'] 
print(f"Retrieved {len(curveballs)} curveballs between 2020-08-01 and 2020-08-07")


100%|██████████| 7/7 [00:12<00:00,  1.82s/it]



Result:
This is a large query, it may take a moment to complete
Retrieved 2025 curveballs between 2020-08-01 and 2020-08-07


*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
# Select key metrics
metrics = ['release_speed', 'release_spin_rate', 'release_pos_x', 'release_pos_z', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z']

# Group by pitcher and calculate mean for each metric
pitcher_avgs = curveballs.groupby('pitcher')[metrics].mean()

print(f"Created dataframe with curveball averages for {len(pitcher_avgs)} pitchers")

Result:
Created dataframe with curveball averages for 215 pitchers


*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
# Scale the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(pitcher_avgs)

# Get Scherzer's data 
scherzer_data = pitcher_avgs.loc[scherzer_id]
scherzer_scaled = scaler.transform(scherzer_data.values.reshape(1,-1))

# Train KNN model
model = NearestNeighbors(n_neighbors=3)
model.fit(scaled_

In [8]:
task = 'yes'
result = baseball_lambda.execute_workflow(task, session_id)
print(result)


*** Entered Orchestrate Node ***

*** Entered Memorize Node ***

Thank you, task has been commited to memory
